In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import difflib
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Input Data

In [24]:

movie_df = pd.read_csv('/Users/nguyenthiduy/Downloads/Khái Phá dữ liệu hiệ đại- MovieLEns/dataset/moviesdata.csv',index_col=0)
movie_df.head(10)

movie_id                                movie_title  rating  \
0    823464            Godzilla x Kong: The New Empire   7.105   
1   1011985                            Kung Fu Panda 4   6.928   
2    359410                                 Road House   7.266   
3    634492                                 Madame Web   5.661   
4    856289  Creation of the Gods I: Kingdom of Storms   7.179   
5    601796                                   Alienoid   7.168   
6    763215                                     Damsel   7.173   
7   1096197                                  No Way Up   6.212   
8    624091                                   Sri Asih   6.182   
9       615                  The Passion of the Christ   7.501   

                                            overview  \
0  Following their explosive showdown, Godzilla a...   
1  Po is gearing up to become the spiritual leade...   
2  Ex-UFC fighter Dalton takes a job as a bouncer...   
3  Forced to confront revelations about her past,...   
4  Based on the most well-known classical fantasy...   
5  In the year 2022, a Robot Guard manages alien ...   
6  A young woman's marriage to a charming prince ...   
7  Characters from different backgrounds are thro...   
8  Alana discover the truth about her origin: she...   
9  A graphic portrayal of the last twelve hours o...   

                                              genres  \
0         ['Action', 'Science Fiction', 'Adventure']   
1  ['Action', 'Adventure', 'Animation', 'Comedy',...   
2                             ['Action', 'Thriller']   
3                              ['Action', 'Fantasy']   
4                       ['Action', 'Fantasy', 'War']   
5  ['Science Fiction', 'Action', 'Fantasy', 'Adve...   
6                 ['Fantasy', 'Action', 'Adventure']   
7                   ['Action', 'Horror', 'Thriller']   
8  ['Action', 'Adventure', 'Science Fiction', 'Fa...   
9                                          ['Drama']   

                     directors  \
0             ['Adam Wingard']   
1            ['Mike Mitchell']   
2               ['Doug Liman']   
3            ['S.J. Clarkson']   
4                 ['Wuershan']   
5           ['Choi Dong-hoon']   
6  ['Juan Carlos Fresnadillo']   
7              ['Claudio Fäh']   
8              ['Upi Avianto']   
9               ['Mel Gibson']   

                                                cast  \
0  ['Rebecca Hall', 'Kaylee Hottle', 'Brian Tyree...   
1  ['Jack Black', 'Awkwafina', 'Bryan Cranston', ...   
2  ['Jake Gyllenhaal', 'Daniela Melchior', 'Conor...   
3  ['Dakota Johnson', 'Sydney Sweeney', 'Isabela ...   
4  ['Kris Phillips', 'Li Xuejian', 'Huang Bo', 'Y...   
5  ['Ryu Jun-yeol', 'Kim Woo-bin', 'Kim Tae-ri', ...   
6  ['Millie Bobby Brown', 'Ray Winstone', 'Angela...   
7  ['Sophie McIntosh', 'Will Attenborough', 'Jere...   
8  ['Pevita Pearce', 'Reza Rahadian', 'Christine ...   
9  ['Jim Caviezel', 'Maia Morgenstern', 'Christo ...   

                                            keywords  \
0  ['giant monster', 'sequel', 'dinosaur', 'kaiju...   
1  ['martial arts', 'kung fu', 'china', 'sequel',...   
2  ['florida keys', 'remake', 'bouncer', 'fightin...   
3  ['new york city', 'superhero', 'spider', 'base...   
4  ['monster', 'based on novel or book', 'mytholo...   
5  ['prisoner', 'time warp', 'alien', 'historical...   
6  ['princess', 'fire', 'chase', 'kingdom', 'swor...   
7  ['ocean', 'bodyguard', 'rescue mission', 'chil...   
8  ['bumi langit', 'jagat sinema bumilangit', 'sr...   
9  ['mission', 'suffering', 'roman empire', 'chri...   

                                        user_ratings original_language  \
0         [{'user_id': 'msbreviews', 'rating': 7.0}]                en   
1  [{'user_id': 'ChrisSawin', 'rating': 6.0}, {'u...                en   
2         [{'user_id': 'msbreviews', 'rating': 6.0}]                en   
3  [{'user_id': 'Geronimo1967', 'rating': 5.0}, {...                en   
4       [{'user_id': 'Geronimo1967', 'rating': 7.0}]           

In [25]:
movie_df.shape

(9711, 15)

In [26]:
movie_df['movie_title'].value_counts()

Godzilla                   5
Alice in Wonderland        4
Beauty and the Beast       4
The Three Musketeers       4
Cinderella                 4
                          ..
Crawlspace                 1
Beverly Hills Chihuahua    1
Kimi                       1
Becoming                   1
Suffragette                1
Name: movie_title, Length: 9382, dtype: int64

In [27]:
movie_df.columns

Index(['movie_id', 'movie_title', 'rating', 'overview', 'genres', 'directors',
       'cast', 'keywords', 'user_ratings', 'original_language', 'duration',
       'release_date', 'revenue', 'is_seen', 'index'],
      dtype='object')

# Feature Selection

In [28]:
#Chọn những thuộc tính ảnh hưởng chính đến hệ thống recommentdation
movie=movie_df[['movie_id',	'movie_title', 'overview', 'genres', 'keywords', 'cast', 'directors']]
movie

movie_id                                movie_title  \
0       823464            Godzilla x Kong: The New Empire   
1      1011985                            Kung Fu Panda 4   
2       359410                                 Road House   
3       634492                                 Madame Web   
4       856289  Creation of the Gods I: Kingdom of Storms   
...        ...                                        ...   
9706    554590                                 Tod@s Caen   
9707    132923                   Guns, Girls and Gambling   
9708     83770                                On the Road   
9709    254869                                 Torrente 5   
9710    245168                                Suffragette   

                                               overview  \
0     Following their explosive showdown, Godzilla a...   
1     Po is gearing up to become the spiritual leade...   
2     Ex-UFC fighter Dalton takes a job as a bouncer...   
3     Forced to confront revelations about her past,...   
4     Based on the most well-known classical fantasy...   
...                                                 ...   
9706  A pair of seducers try to prove who has the be...   
9707  The story throws Elvis impersonators, Indians,...   
9708  Dean and Sal are the portrait of the Beat Gene...   
9709  Crooked cop Torrente gets out of jail in the y...   
9710  Based on true events about the foot soldiers o...   

                                                 genres  \
0            ['Action', 'Science Fiction', 'Adventure']   
1     ['Action', 'Adventure', 'Animation', 'Comedy',...   
2                                ['Action', 'Thriller']   
3                                 ['Action', 'Fantasy']   
4                          ['Action', 'Fantasy', 'War']   
...                                                 ...   
9706                                         ['Comedy']   
9707                              ['Crime', 'Thriller']   
9708                             ['Adventure', 'Drama']   
9709                               ['Action', 'Comedy']   
9710                               ['Drama', 'History']   

                                               keywords  \
0     ['giant monster', 'sequel', 'dinosaur', 'kaiju...   
1     ['martial arts', 'kung fu', 'china', 'sequel',...   
2     ['florida keys', 'remake', 'bouncer', 'fightin...   
3     ['new york city', 'superhero', 'spider', 'base...   
4     ['monster', 'based on novel or book', 'mytholo...   
...                                                 ...   
9706                                                 []   
9707  ['sheriff', 'casino', 'apache nation', 'native...   
9708  ['friendship', 'based on novel or book', 'fune...   
9709  ['future', 'robbery', 'casino', 'anti hero', '...   
9710  ['london, england', 'biography', 'feminist', '...   

                                                   cast            directors  
0     ['Rebecca Hall', 'Kaylee Hottle', 'Brian Tyree...     ['Adam Wingard']  
1     ['Jack Black', 'Awkwafina', 'Bryan Cranston', ...    ['Mike Mitchell']  
2     ['Jake Gyllenhaal', 'Daniela Melchior', 'Conor...       ['Doug Liman']  
3     ['Dakota Johnson', 'Sydney Sweeney', 'Isabela ...    ['S.J. Clarkson']  
4     ['Kris Phillips', 'Li Xuejian', 'Huang Bo', 'Y...         ['Wuershan']  
...                                                 ...                  ...  
9706  ['Martha Higareda', 'Omar Chaparro', 'Claudia ...   ['Ariel Winograd']  
9707  ['Gary Oldman', 'Christian Slater', 'Dane Cook...  ['Michael Winnick']  
9708  ['Garrett Hedlund', 'Sam Riley', 'Kristen Stew...    ['Walter Salles']  
9709  ['Santiago Segura', 'Julián López', 'Jesulín d...  ['Santiago Segura']  
9710  ['Carey Mulligan', 'Helena Bonham Carter', 'Br...     ['Sarah Gavron']  

[9711 rows x 7 columns]

# datapreprocessing


In [29]:
movie.isnull().sum()

movie_id        0
movie_title     0
overview       47
genres          0
keywords        0
cast            0
directors       0
dtype: int64

In [30]:

for feature in movie:
  movie[feature] = movie[feature].fillna('')

/var/folders/kp/zhftn9l95477nfk6phfjh8wh0000gn/T/ipykernel_2590/86778764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie[feature] = movie[feature].fillna('')


In [31]:
movie.isnull().sum()

movie_id       0
movie_title    0
overview       0
genres         0
keywords       0
cast           0
directors      0
dtype: int64

In [32]:
movie.duplicated().sum()

0

In [33]:
movie_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9711 entries, 0 to 9710
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           9711 non-null   int64  
 1   movie_title        9711 non-null   object 
 2   rating             9711 non-null   float64
 3   overview           9664 non-null   object 
 4   genres             9711 non-null   object 
 5   directors          9711 non-null   object 
 6   cast               9711 non-null   object 
 7   keywords           9711 non-null   object 
 8   user_ratings       9711 non-null   object 
 9   original_language  9711 non-null   object 
 10  duration           9711 non-null   int64  
 11  release_date       9689 non-null   object 
 12  revenue            9711 non-null   int64  
 13  is_seen            9711 non-null   bool   
 14  index              9711 non-null   int64  
dtypes: bool(1), float64(1), int64(4), object(9)
memory usage: 1.1+ MB


In [34]:
movie.head()


movie_id                                movie_title  \
0    823464            Godzilla x Kong: The New Empire   
1   1011985                            Kung Fu Panda 4   
2    359410                                 Road House   
3    634492                                 Madame Web   
4    856289  Creation of the Gods I: Kingdom of Storms   

                                            overview  \
0  Following their explosive showdown, Godzilla a...   
1  Po is gearing up to become the spiritual leade...   
2  Ex-UFC fighter Dalton takes a job as a bouncer...   
3  Forced to confront revelations about her past,...   
4  Based on the most well-known classical fantasy...   

                                              genres  \
0         ['Action', 'Science Fiction', 'Adventure']   
1  ['Action', 'Adventure', 'Animation', 'Comedy',...   
2                             ['Action', 'Thriller']   
3                              ['Action', 'Fantasy']   
4                       ['Action', 'Fantasy', 'War']   

                                            keywords  \
0  ['giant monster', 'sequel', 'dinosaur', 'kaiju...   
1  ['martial arts', 'kung fu', 'china', 'sequel',...   
2  ['florida keys', 'remake', 'bouncer', 'fightin...   
3  ['new york city', 'superhero', 'spider', 'base...   
4  ['monster', 'based on novel or book', 'mytholo...   

                                                cast          directors  
0  ['Rebecca Hall', 'Kaylee Hottle', 'Brian Tyree...   ['Adam Wingard']  
1  ['Jack Black', 'Awkwafina', 'Bryan Cranston', ...  ['Mike Mitchell']  
2  ['Jake Gyllenhaal', 'Daniela Melchior', 'Conor...     ['Doug Liman']  
3  ['Dakota Johnson', 'Sydney Sweeney', 'Isabela ...  ['S.J. Clarkson']  
4  ['Kris Phillips', 'Li Xuejian', 'Huang Bo', 'Y...       ['Wuershan']

In [35]:
combined_features = movie['genres']+' '+movie['cast']+' '+movie['keywords']+' '+movie['directors']+movie['movie_title']
print(combined_features)

0       ['Action', 'Science Fiction', 'Adventure'] ['R...
1       ['Action', 'Adventure', 'Animation', 'Comedy',...
2       ['Action', 'Thriller'] ['Jake Gyllenhaal', 'Da...
3       ['Action', 'Fantasy'] ['Dakota Johnson', 'Sydn...
4       ['Action', 'Fantasy', 'War'] ['Kris Phillips',...
                              ...                        
9706    ['Comedy'] ['Martha Higareda', 'Omar Chaparro'...
9707    ['Crime', 'Thriller'] ['Gary Oldman', 'Christi...
9708    ['Adventure', 'Drama'] ['Garrett Hedlund', 'Sa...
9709    ['Action', 'Comedy'] ['Santiago Segura', 'Juli...
9710    ['Drama', 'History'] ['Carey Mulligan', 'Helen...
Length: 9711, dtype: object


In [36]:
combined_features

0       ['Action', 'Science Fiction', 'Adventure'] ['R...
1       ['Action', 'Adventure', 'Animation', 'Comedy',...
2       ['Action', 'Thriller'] ['Jake Gyllenhaal', 'Da...
3       ['Action', 'Fantasy'] ['Dakota Johnson', 'Sydn...
4       ['Action', 'Fantasy', 'War'] ['Kris Phillips',...
                              ...                        
9706    ['Comedy'] ['Martha Higareda', 'Omar Chaparro'...
9707    ['Crime', 'Thriller'] ['Gary Oldman', 'Christi...
9708    ['Adventure', 'Drama'] ['Garrett Hedlund', 'Sa...
9709    ['Action', 'Comedy'] ['Santiago Segura', 'Juli...
9710    ['Drama', 'History'] ['Carey Mulligan', 'Helen...
Length: 9711, dtype: object

# Feature Extraction

In [37]:
# chuyển đổi văn bản dạng tho sang dạng ma trận TF-IDF
vectorizer=TfidfVectorizer()
#tính ra các gia trị TF-IDF của từng từ trong văn bản đó
feature_vectors=vectorizer.fit_transform(combined_features)
#ma trận thưa- sparse Matrix
print(feature_vectors)

  (0, 9632)	0.1588799455873867
  (0, 22243)	0.09814324248180044
  (0, 31295)	0.05394647461989754
  (0, 33982)	0.2072452104209781
  (0, 408)	0.12780696270423025
  (0, 10058)	0.19577615360228168
  (0, 17104)	0.3118443804848138
  (0, 16832)	0.12753796636410453
  (0, 21243)	0.2177702390407544
  (0, 12230)	0.3439823142056167
  (0, 1542)	0.18072527466158406
  (0, 34171)	0.10456846880120706
  (0, 10322)	0.07645899498454942
  (0, 16123)	0.16325703954403262
  (0, 8378)	0.1660509071255755
  (0, 27992)	0.10229603118400712
  (0, 21241)	0.12357479399577435
  (0, 11992)	0.27275869637025096
  (0, 10573)	0.24628792292783555
  (0, 888)	0.13694987476711878
  (0, 29898)	0.1705486796562778
  (0, 7508)	0.14133874369086458
  (0, 13695)	0.1383319913240066
  (0, 32362)	0.19577615360228168
  (0, 4373)	0.12991852586028477
  :	:
  (9710, 11739)	0.26020365097283077
  (9710, 10531)	0.24998963125947177
  (9710, 8969)	0.23119795102607013
  (9710, 30229)	0.2757103776009432
  (9710, 30046)	0.20581263576138684
  (9710,

In [38]:
feature_vectors.shape

(9711, 35216)

In [39]:
# Tra cứu từ từ chỉ số 9632
word = vectorizer.get_feature_names_out()[9632]
print(word)


empire


In [40]:
#Lấy các giá trị giống nhau sử dụng cosine similarity
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.02157211 0.00381547 ... 0.01032211 0.01452811 0.        ]
 [0.02157211 1.         0.00445335 ... 0.00532643 0.02089652 0.        ]
 [0.00381547 0.00445335 1.         ... 0.05885587 0.00399659 0.        ]
 ...
 [0.01032211 0.00532643 0.05885587 ... 1.         0.         0.00584728]
 [0.01452811 0.02089652 0.00399659 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.00584728 0.         1.        ]]


In [41]:
similarity.shape

(9711, 9711)

In [42]:


import difflib

def recommend(movie_df, similarity, movie_name):
    list_of_all_titles = movie_df['movie_title'].tolist()
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    
    if find_close_match:
        close_match = find_close_match[0]
        index_of_the_movie = movie_df[movie_df.movie_title == close_match].index[0]
        #Tạo một danh sách các tuple chứa chỉ số phim và điểm tương đồng của nó với phim đã chọn.
        similarity_score = list(enumerate(similarity[index_of_the_movie]))
        sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        return sorted_similar_movies[:10]  # Chỉ lấy 10 bộ phim đầu tiên
    else:
        print("Không tìm thấy phim gần giống với tên đã nhập.")
        return []

def print_recommendations(movie_df, sorted_similar_movies):
    for i, movie in enumerate(sorted_similar_movies, start=1):
        index = movie[0]
        title_from_index = movie_df.loc[index, 'movie_title']
        print(f"{i}. {title_from_index}")
        if i >= 10:
            break

# Sử dụng hàm để gợi ý các bộ phim tương tự và in ra kết quả
movie_name = input('Nhập tên phim yêu thích của bạn: ')
sorted_similar_movies = recommend(movie_df, similarity, movie_name)
print_recommendations(movie_df, sorted_similar_movies)


1. Kung Fu Panda
2. Kung Fu Panda 3
3. Kung Fu Panda 2
4. Kung Fu Panda 4
5. Kung Fu Panda: Secrets of the Furious Five
6. Kung Fu Panda Holiday
7. Kung Fu Panda: Secrets of the Masters
8. Kung Fu Panda: Secrets of the Scroll
9. Once Upon a Time in China II
10. Once Upon a Time in China III


In [45]:

import pickle
pickle.dump(movie,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))


In [44]:
import pandas as pd

print("Phiên bản của Pandas:", pd.__version__)


Phiên bản của Pandas: 1.5.3
